In [ ]:
# pip install --upgrade python-telegram-bot

In [12]:
import logging
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes
from google.colab import drive
import os

In [16]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
model_path = '/content/drive/MyDrive/AI_chatbot(4th_prj)/sample_transformer_model.pth'


In [ ]:
TOKEN = 'NEW'

# 로깅 설정
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# 모델과 토크나이저 로드
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# 응답 생성 함수
def generate_response(question, tokenizer, model, max_length=512):
    inputs = tokenizer.encode(question, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# /start 명령어 처리 함수
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user = update.effective_user
    await update.message.reply_text(f'안녕하세요 {user.first_name}!\n질문을 입력해 주세요.')

# /help 명령어 처리 함수
async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text('도움이 필요하시면 이 봇에게 질문을 하세요.')

# 메시지 처리 함수
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user_input = update.message.text
    response = generate_response(user_input, tokenizer, model)
    await update.message.reply_text(response)

# 메인 함수
def main() -> None:
    # 애플리케이션 빌더 생성
    app = ApplicationBuilder().token(TOKEN).build()

    # 핸들러 등록
    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("help", help_command))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    # 봇 시작
    app.run_polling()

if __name__ == '__main__':
    main()